In [1]:
import pandas as pd
sms = pd.read_csv("Sams2/TRAIN_info_SMS.csv",encoding='latin-1')
new_sms = pd.read_csv("Sams2/DEV_info_SMS.csv",encoding='latin-1')
sms.shape

(12000, 2)

In [2]:
sms.head()

,Label,Message
0,Delivery,"Dear Guest, Thanks for choosing The Edison. Or..."
1,Hotel,Welcome to Southern University at Shreveport R...
2,Payment,Bill dated 2013-01-14 for Rs 626 has been gene...
3,Payment,Bill dated 2014-11-30 for Rs 323 has been gene...
4,Appointment,"DEAR Zachary, THANK YOU FOR YOUR BOOKING (ORDE..."


In [3]:
for col in sms.columns:
    if 'Unnamed' in col:
        del sms[col]

In [4]:
sms.head()

,Label,Message
0,Delivery,"Dear Guest, Thanks for choosing The Edison. Or..."
1,Hotel,Welcome to Southern University at Shreveport R...
2,Payment,Bill dated 2013-01-14 for Rs 626 has been gene...
3,Payment,Bill dated 2014-11-30 for Rs 323 has been gene...
4,Appointment,"DEAR Zachary, THANK YOU FOR YOUR BOOKING (ORDE..."


In [5]:
sms = sms.rename(columns={'v1': 'label', 'v2': 'message'})
sms.head()

,Label,Message
0,Delivery,"Dear Guest, Thanks for choosing The Edison. Or..."
1,Hotel,Welcome to Southern University at Shreveport R...
2,Payment,Bill dated 2013-01-14 for Rs 626 has been gene...
3,Payment,Bill dated 2014-11-30 for Rs 323 has been gene...
4,Appointment,"DEAR Zachary, THANK YOU FOR YOUR BOOKING (ORDE..."


In [6]:
sms.shape

(12000, 2)

In [7]:
sms.Label.value_counts()

Expiry         1000
PickUp         1000
Flight         1000
Appointment    1000
Bus            1000
Hotel          1000
Cab            1000
Reservation    1000
Train          1000
Payment        1000
Delivery       1000
Movie          1000
Name: Label, dtype: int64

In [8]:
sms['label_num']=sms.Label.map({"Expiry":0,"PickUp":1,"Flight":2,"Appointment":3,"Bus":4, "Hotel":5,"Cab":6,"Reservation":7,"Train":8,"Payment":9,"Delivery":10,"Movie":11})
# checking the conversion
sms.head()

,Label,Message,label_num
0,Delivery,"Dear Guest, Thanks for choosing The Edison. Or...",10
1,Hotel,Welcome to Southern University at Shreveport R...,5
2,Payment,Bill dated 2013-01-14 for Rs 626 has been gene...,9
3,Payment,Bill dated 2014-11-30 for Rs 323 has been gene...,9
4,Appointment,"DEAR Zachary, THANK YOU FOR YOUR BOOKING (ORDE...",3


In [9]:
x = sms.Message
x_new = new_sms.Message
y = sms.label_num
print(x.shape)
print(y.shape)

(12000,)
(12000,)


In [10]:
# split x and y into traing and testing set
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=1)
print(x_train.shape)
print(x_test.shape)

(9000,)
(3000,)


In [11]:
# import and instatiate the countvectorizer with default parameter
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
# Creating a document term matrix
x_train_dtm = vect.fit_transform(x_train)
x_train_dtm

<9000x18933 sparse matrix of type '<class 'numpy.int64'>'
	with 253412 stored elements in Compressed Sparse Row format>

In [12]:
# Transform testing data into document term matrix
x_test_dtm = vect.transform(x_test)
x_test_dtm

<3000x18933 sparse matrix of type '<class 'numpy.int64'>'
	with 79869 stored elements in Compressed Sparse Row format>

In [13]:
# Import and instatiate the naive bayes model
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [14]:
# Train the model using x_train_dtm
%time nb.fit(x_train_dtm,y_train)
# %time model.fit(x_train_dtm,y_train)

Wall time: 31 ms


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [17]:
y_pred_class = nb.predict(vect.transform(x_test))

In [19]:
# make class prediction for x_test_dtm
y_pred_class = nb.predict(vect.transform(x_new))

In [20]:
y_pred_class_df =  pd.DataFrame(y_pred_class)

In [21]:
y_pred_class_df.columns = ["Label"]

In [23]:
y_pred_class_final = y_pred_class_df.Label.map({0:"Expiry",1:"PickUp",2:"Flight",3:"Appointment",4:"Bus",5:"Hotel",6:"Cab",7:"Reservation",8:"Train",9:"Payment",10:"Delivery",11:"Movie"})

In [25]:
y_pred_class_final

0            PickUp
1           Payment
2           Payment
3            Flight
4             Hotel
5       Appointment
6             Hotel
7             Train
8             Movie
9       Reservation
10      Reservation
11              Cab
12           PickUp
13              Bus
14         Delivery
15            Hotel
16              Cab
17            Movie
18      Appointment
19           Flight
20           Expiry
21           Flight
22              Cab
23           Flight
24      Reservation
25           Expiry
26          Payment
27           PickUp
28              Bus
29           Flight
           ...     
2970         Expiry
2971          Hotel
2972        Payment
2973          Movie
2974         PickUp
2975    Reservation
2976          Movie
2977        Payment
2978    Reservation
2979         PickUp
2980         Expiry
2981            Cab
2982        Payment
2983         Expiry
2984    Reservation
2985          Hotel
2986            Cab
2987            Bus
2988            Cab


In [26]:
results = pd.DataFrame(y_pred_class_final)
results.to_csv('results_info.csv')

In [27]:
# calculate the accuarcy of class prediction
from sklearn import metrics
metrics.accuracy_score(y_test,y_pred_class)

0.085000000000000006

In [ ]:
y_pred_class = nb.predict()